# Hybrid Approximate Nearest Neighbors Algorithm for Movie Recommendations

## By Cody Zupnick, Kathy Lin and Priya Medberry

Our objective will be to maximize the accuracy of the recommendations provided to Movielens users when considering the results of recommending the Top-5 movies for a given user. For this purpose, we will be using a dataset from Movielens with over 10 million ratings. This notebook contains 5 sections: 
1.  Business Objectives/Background
2.  Exploratory Data Analysis
3.  Motivation for Hybrid Model
4.  Implementation of Hybrid Model
5.  Performance of Hybrid Model


Firstly, we'll provide some background on the business objectives of Movielens. Then we'll dive into the dataset and conduct some exploratory data analysis. Then we will describe the motivation and implementation of a hybrid model used to produce these recommendations. Lastly, we will compare the performance of our designed hybrid model against the performance of prior models such as a raw item-based collaborative filtering model, and the matrix factorization model. 

### Background
 MovieLens is a web-based recommender system and virtual community that recommends movies for its users to watch, based on their film preferences using CF of members' movie ratings and reviews. As with any recommendation system, the business objective is to build trust between the consumer and the recommendation system by providing accurate predictions of items that a consumer may like based on history of user interactions with the system (both implicitly and explicitly). For the purposes of this dataset, we'll be focusing solely on explicit ratings provided in the past by the users. 
 
 However, it is important to note early on that there are often 3 types of common challenges with implementing these algorithms for the purpose of provide accurate recommendations: time, scale and quality. Although it is nice to produce accurate recommendations in a short period of time given large numbers of users and movies, there is a tradeoff between computation time and the number of movies / users in the system. Likewise, the faster an algorithm runs, which is also known as more scalability, the worse quality recommendations it usually provides. Therefore, we have designed in this project a potential solution to this tradeoff triangle. We'd still like to maximize accuracy in a reasonable period of time for a large number of users and movies. Therefore, we will use a hybrid model of approximate nearest neighbors method (item-based collaborative filtering method) with a dimensionality reducing algorithm. This hybrid model will be able to counteract the problem of scalability.